# Theme Park Review Analysis

## by Justin Sierchio

In this analysis, we will be looking at reviews from 3 Disneyland(R) parks in California, Paris and Hong Kong as they were posted on Trip Advisor. We would like to be able to answer the following questions:

<ul>
    <li>Which park had the best reviews?</li>
    <li>What was the most common complaint?</li>
    <li>What are some other conclusions we might able to draw from this analysis?</li>
</ul>

This data is in .csv file format and is from Kaggle at: https://www.kaggle.com/arushchillar/disneyland-reviews/download. More information related to the dataset can be found at: https://www.kaggle.com/arushchillar/disneyland-reviews.

## Notebook Initialization

In [1]:
# Import Relevant Libraries
import pandas as pd
import numpy as np
import seaborn as sns 
import matplotlib.pyplot as plt
import datetime

print('Initial libraries loaded into workspace!')

Initial libraries loaded into workspace!


In [2]:
# Upload Datasets for reviews
df_REVIEWS = pd.read_csv("DisneylandReviews.csv");

print('Datasets uploaded!');

Datasets uploaded!


In [3]:
# Display 1st 5 rows from Theme Park review set
df_REVIEWS.head()

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong


Let's briefly list the column titles and their descriptions:

<ul>
    <li> Review_ID: ID given to each entry </li>
    <li> Rating: range of 1 to 5, 1 being bad and 5 being great </li>
    <li> Year_Month: date when reviewer visited park </li>
    <li> Reviewer_Location: where reviewed is located </li>
    <li> Review_Text: visitor comments </li>
    <li> Disneyland_Branch: which location of Disneyland Park reviewed?</li>
</ul>